# Unsupervised training of the encoder layers

## Initial setup
### Imports

In [10]:
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
from datetime import datetime
from models import Autoencoder, EncoderStack
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import normalize
import dataset
import neptune
import neptune_tensorboard as neptune_tb
import os
import logging

2.2.0-dlenv


### Set experiment configuration

In [11]:
config = {
    "N_NODES": [2000, 1000, 500],
    "DROPOUT": [0.1],
    "BATCH_SIZE": 15,
    "EPOCHS": 100,
    "TEST_RATIO": 0.20,
    "DATA_BUCKET": "sdae-geo",
    "DATA_OBJECT": "GEO_data_batch_corr_final.csv",
    "DATA_LABELS": " GBM_class.csv",
    "VERBOSITY": 2,
    "LOG_DIR": "./log_dir",
    "PATIENCE":3
}


## Initialize Netptune and Tensorboard logging

In [12]:
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYjNiYmZhYjEtNzc3ZS00Y2NhLWI5NTgtYWU0MmQyMWJhM2I0In0="
os.environ['NEPTUNE_PROJECT']='AiPBBAND/SDAE'
os.environ['NEPTUNE_NOTEBOOK_ID']="ecd86e96-4da7-44e3-9a17-43da2dfcae35"
os.environ['NEPTUNE_NOTEBOOK_PATH']="constrained-SDAE/sdae.ipynb"

neptune.init(os.environ['NEPTUNE_PROJECT'], api_token=os.environ['NEPTUNE_API_TOKEN'])

logger = logging.getLogger("SDAE")
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

experiment = neptune.create_experiment(name='configuration', params=config, 
    logger=logger, source=["dataset.py", "models.py"])

os.environ['EXP_DIR'] = os.path.join(config["LOG_DIR"], experiment.id)
os.mkdir(os.environ['EXP_DIR'])

logger.info("project directory: {}".format(os.environ['EXP_DIR']))
!neptune tensorboard ${EXP_DIR} --project ${NEPTUNE_PROJECT}
%load_ext tensorboard

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/AiPBBAND/SDAE/e/SAN-2


project directory: ./log_dir/SAN-2
project directory: ./log_dir/SAN-2


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Start tensorboard server
Tensorboard by running the following command in a terminal:

In [13]:
print("tensorboard --logdir {} --bind_all".format(os.environ['EXP_DIR']))

tensorboard --logdir ./log_dir/SAN-2 --bind_all


**Tensorboard cannot server over HTTPS, use external HTTP url: http://34.77.45.86:6006/**

# Load and preprocess the data

## Load data from Google Storage

In [14]:
dataframe = dataset.load_gs_data(config['DATA_BUCKET'], config['DATA_OBJECT'], os.environ['EXP_DIR'])

Initializing dataset...
Initializing dataset...
File GEO_data_batch_corr_final.csv downloaded to ./log_dir/SAN-2/GEO_data_batch_corr_final.csv.
File GEO_data_batch_corr_final.csv downloaded to ./log_dir/SAN-2/GEO_data_batch_corr_final.csv.
Loaded 954 samples with 6785 features.
Loaded 954 samples with 6785 features.


## Normalize data feature-wize

In [15]:
data = dataframe.values
data = normalize(data)

## Split into training and testing sets

In [16]:
rs = ShuffleSplit(n_splits=1, test_size=config['TEST_RATIO'], random_state=0)
split_itterator = rs.split(data)
i_train, i_test = next(split_itterator)
train_path = os.path.join(os.environ['EXP_DIR'], "train_indices.npy")
test_path = os.path.join(os.environ['EXP_DIR'], "test_indices.npy")
np.save(train_path, i_train)
np.save(test_path, i_test)
neptune.log_artifact(train_path)
neptune.log_artifact(test_path)

x_train, x_test = data[i_train], data[i_test]

logger.info("{}% of samples for training: {} training, {} testing.".format(int(config['TEST_RATIO']*100), len(i_train), len(i_test)))

20% of samples for training: 763 training, 191 testing.
20% of samples for training: 763 training, 191 testing.


# Train encoder layers

In [17]:
logger.info("Starting training for {} encoder layers.".format(len(config["N_NODES"])))

x_train_out, x_test_out = x_train, x_test
for idx, num_hidden in enumerate(config["N_NODES"]):
    logger.info("Training layer {} with {} hidden nodes..".format(idx, num_hidden))
    encoder = Autoencoder(x_train_out.shape[1], num_hidden, os.environ['EXP_DIR'])
    
    recon_mse = encoder.fit(x_train_out, x_test_out, batch_size=config["BATCH_SIZE"], 
        num_epochs=config["EPOCHS"], verbose=config["VERBOSITY"], patience=config["PATIENCE"])
    
    x_train_out = encoder.encoder_model.predict(x_train_out)
    x_test_out = encoder.encoder_model.predict(x_test_out)
    logger.info("Training losss for layer {}: {} ".format(idx, recon_mse[0]))
    logger.info("Testing loss for layer {}: {} ".format(idx, recon_mse[1]))
    
    model_path = os.path.join(os.environ['EXP_DIR'], "encoders", "model_{}_{}".format(idx,num_hidden))
    encoder.encoder_model.save(model_path)
    neptune.log_artifact(model_path)
    neptune.set_property('encoder{}'.format(idx), model_path)




Starting training for 3 encoder layers.
Starting training for 3 encoder layers.
Training layer 0 with 2000 hidden nodes..
Training layer 0 with 2000 hidden nodes..


Epoch 1/100
51/51 [==============================] - 11s 222ms/step - loss: 0.1742 - val_loss: 0.0683
Epoch 2/100
51/51 [==============================] - 11s 215ms/step - loss: 0.0202 - val_loss: 0.0057
Epoch 3/100
51/51 [==============================] - 11s 215ms/step - loss: 0.0029 - val_loss: 0.0035
Epoch 4/100
51/51 [==============================] - 11s 213ms/step - loss: 0.0011 - val_loss: 7.6653e-04
Epoch 5/100
51/51 [==============================] - 11s 219ms/step - loss: 4.6854e-04 - val_loss: 3.2537e-04
Epoch 6/100
51/51 [==============================] - 11s 213ms/step - loss: 2.5074e-04 - val_loss: 1.8888e-04
Epoch 7/100
51/51 [==============================] - 11s 217ms/step - loss: 1.4555e-04 - val_loss: 1.1732e-04
Epoch 8/100
51/51 [==============================] - 11s 211ms/step - loss: 9.4232e-05 - val_loss: 7.2665e-05
Epoch 9/100
51/51 [==============================] - 11s 218ms/step - loss: 6.3726e-05 - val_loss: 4.9887e-05
Epoch 10/100
51/51 [==================

Trained model saved at: ./log_dir/SAN-2/autoencoder-2000
Trained model saved at: ./log_dir/SAN-2/autoencoder-2000
Training losss for layer 0: 4.2964029489667155e-06 
Training losss for layer 0: 4.2964029489667155e-06 
Testing loss for layer 0: 4.309176347305765e-06 
Testing loss for layer 0: 4.309176347305765e-06 


INFO:tensorflow:Assets written to: ./log_dir/SAN-2/encoders/model_0_2000/assets


Training layer 1 with 1000 hidden nodes..
Training layer 1 with 1000 hidden nodes..


...

INFO:tensorflow:Assets written to: ./log_dir/SAN-2/encoders/model_1_1000/assets


Training layer 2 with 500 hidden nodes..
Training layer 2 with 500 hidden nodes..


Epoch 1/100
51/51 [==============================] - 1s 28ms/step - loss: 0.0171 - val_loss: 0.0106
Epoch 2/100
51/51 [==============================] - 1s 18ms/step - loss: 0.0095 - val_loss: 0.0068
Epoch 3/100
51/51 [==============================] - 1s 19ms/step - loss: 0.0062 - val_loss: 0.0052
Epoch 4/100
51/51 [==============================] - 1s 19ms/step - loss: 0.0042 - val_loss: 0.0036
Epoch 5/100
51/51 [==============================] - 1s 19ms/step - loss: 0.0029 - val_loss: 0.0023
Epoch 6/100
51/51 [==============================] - 1s 19ms/step - loss: 0.0020 - val_loss: 0.0012
Epoch 7/100
51/51 [==============================] - 1s 20ms/step - loss: 0.0014 - val_loss: 0.0010
Epoch 8/100
51/51 [==============================] - 1s 20ms/step - loss: 9.6916e-04 - val_loss: 7.9721e-04
Epoch 9/100
51/51 [==============================] - 1s 19ms/step - loss: 6.9494e-04 - val_loss: 5.9070e-04
Epoch 10/100
51/51 [==============================] - 1s 22ms/step - loss: 5.1584e-0

Trained model saved at: ./log_dir/SAN-2/autoencoder-500
Trained model saved at: ./log_dir/SAN-2/autoencoder-500
Training losss for layer 2: 4.848092430620454e-05 
Training losss for layer 2: 4.848092430620454e-05 
Testing loss for layer 2: 4.881304994341917e-05 
Testing loss for layer 2: 4.881304994341917e-05 


INFO:tensorflow:Assets written to: ./log_dir/SAN-2/encoders/model_2_500/assets


In [18]:
neptune.stop()